In [189]:
import cv2
import sys 
import pandas as pd
import pickle

class SpeedAndDistance_Estimator():
    def __init__(self):
        # frame_window: 몇 개의 프레임 단위로 묶어서 처리할지 설정
        # frame_rate: 초당 프레임 수
        
        self.frame_window = 5
        self.frame_rate = 10
    
    def add_speed_and_distance_to_tracks(self, tracks):
        # 객체와 트랙 정보를 이용하여 주어진 프레임 윈도우 내에서 속도와 이동 거리를 계산
        # 바운딩 박스를 이용해 객체의 시작과 끝 위치를 구하고, 이를 실제 거리 단위로 변환한 후 속도와 거리를 계산하여 트랙에 추가
        
        self.tracks = tracks
        total_distance = {}
       
        for object, object_tracks in tracks.items():
            if object == "ball":
                continue 
            number_of_frames = len(object_tracks)
            for frame_num in range(0, number_of_frames, self.frame_window):
                last_frame = min(frame_num + self.frame_window, number_of_frames - 1)

                for track_id, _ in object_tracks[frame_num].items():
                    if track_id not in object_tracks[last_frame]:
                        continue

                    start_position = object_tracks[frame_num][track_id]['coord_tr']
                    end_position = object_tracks[last_frame][track_id]['coord_tr']
                    
                    if start_position is None or end_position is None:
                        continue

                    # 픽셀 단위를 실제 거리 단위로 변환
                    distance_covered_x = (end_position[0] - start_position[0])/40
                    distance_covered_y = (end_position[1] - start_position[1])/40
                    distance_covered = (distance_covered_x**2 + distance_covered_y**2)**0.5 # 미터 단위

                    time_elapsed = (last_frame - frame_num) / self.frame_rate
                    speed_meters_per_second = distance_covered / time_elapsed
                    speed_km_per_hour = (speed_meters_per_second)*3.6

                    if object not in total_distance:
                        total_distance[object] = {}
                    
                    if track_id not in total_distance[object]:
                        total_distance[object][track_id] = 0
                    
                    total_distance[object][track_id] += distance_covered

                    for frame_num_batch in range(frame_num, last_frame):
                        if track_id not in tracks[object][frame_num_batch]:
                            continue
                        tracks[object][frame_num_batch][track_id]['speed'] = speed_km_per_hour
                        tracks[object][frame_num_batch][track_id]['distance'] = total_distance[object][track_id]
        return(tracks)

In [190]:
# pickle 파일 불러오기
file_path = r"C:\Users\Playdata\Desktop\passmap\tracks_name+ba.pkl"

try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    # print(data)
except FileNotFoundError:
    print(f"Error: {file_path} 파일을 찾을 수 없습니다.")
except pickle.UnpicklingError:
    print("Error: 파일을 불러오는 중에 문제가 발생했습니다.")

estimator = SpeedAndDistance_Estimator()

In [191]:
data2 = estimator.add_speed_and_distance_to_tracks(data)

In [197]:
players_speed_distance = []
len(data2['players'])
for frame_num in range(len(data2['players'])-1):
    for player_id, player_info in data2['players'][frame_num].items():
        players_speed_distance.append({
                'frame': data['players'].index(frame),
                'player_initial': player_id,
                'player_name': player_info['kr_name'],
                'team': player_info['team'],
                'player_speed(km/h)': player_info['speed'],
                'distance(m)': player_info['distance'],
            })

In [198]:
hi = pd.DataFrame(players_speed_distance)

In [199]:
hi

,frame,player_initial,player_name,team,player_speed(km/h),distance(m)
0,1203,OJY,오지영,1,2.276840,0.316228
1,1203,SMJ,손명준,1,0.000000,0.000000
2,1203,KSW,김성욱,1,0.000000,0.000000
3,1203,SDG,신대근,1,3.219938,0.447214
4,1203,SHW,서한울,0,0.180000,0.025000
...,...,...,...,...,...,...
9619,1203,SMJ,손명준,1,4.210701,181.343292
9620,1203,KYS,가윤성,0,1.500000,87.266904
9621,1203,OJY,오지영,1,11.200893,199.605675
9622,1203,SDG,신대근,1,8.828363,246.208346


In [215]:
pass_player_name = hi['player_name'].unique().tolist()
player_stat = []
for player_name in pass_player_name:
    player_team = hi[hi['player_name'] == player_name]['team'].tolist()[0]
    speed_data = hi[hi['player_name'] == player_name]['player_speed(km/h)'].tolist()
    speed_average = sum(speed_data)/len(speed_data)
    speed_max = max(speed_data)/1.5 # 영상을 10 frame 단위로 사용하여, 보정값 1.5로 나누어줌
    distance = round(max(hi[hi['player_name'] == player_name]['distance(m)'].tolist()),2)
    player_stat.append({
        'player_name': player_name,
        'team': player_team,
        'average_speed(km/h)': speed_average,
        'max_speed(km/h)': speed_max,
        'total_distance(m)': distance,
        })
df_hi = pd.DataFrame(player_stat)

In [216]:
df_hi

,player_name,team,average_speed(km/h),max_speed(km/h),total_distance(m)
0,오지영,1,5.973237,21.148957,199.61
1,손명준,1,5.426732,11.476167,181.34
2,김성욱,1,1.123080,7.903619,37.53
3,신대근,1,7.367831,26.986930,246.21
4,서한울,0,4.976710,25.615152,166.31
5,김성일,0,6.192091,35.537226,206.92
6,강동욱,0,4.936110,12.310971,164.95
7,가윤성,0,2.611478,34.461526,87.27
